In [1]:
import sys, datetime
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel
from PyQt5.QtCore import QTimer

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

#FigureCanvas, Figure 嵌入pyqt5之matplotlib套件
    #因嵌入matplotlib, 不使用排版而直接使用pixel定位, PushButton要多self參數
#QTimer(class) 回傳一個timer物件
    #.timer.timeout.connect(func) class中之func多一個thread
    #.timer.start(1000/fps) 多久更新一次畫面
#QMainWindow 如同Widget
    #.button.clicked.connect 也是多一個thread

In [2]:
flag = False
L = []

class App(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.show()
    def initUI(self):
        self.setWindowTitle('Vibration signal')
        self.setGeometry(100, 200, 1640, 400)
        # canvas thread
        obj = PlotCanvas(self, width=15, height=4)
        self.timer = QTimer(obj)
        self.timer.timeout.connect(obj.plotc)
        self.FRAME_PER_SECOND = 1
        self.timer.start(1000/self.FRAME_PER_SECOND)
        # collect button thread
        self.button = QPushButton(self)
        self.button.setText("collect data")
        self.button.move(1500,0)
        self.button.resize(160,100)
        self.button.clicked.connect(self.detect)
        
        self.label = QLabel(self)
        self.label.setText("Data is not collected")
        self.label.move(1500,100)
        self.label.resize(160,100)
        # save button thread
        self.button2 = QPushButton(self)
        self.button2.setText("save data \n and clear collected")
        self.button2.move(1500,200)
        self.button2.resize(160,100)
        self.button2.clicked.connect(self.save)
    
    def detect(self):
        global flag
        if not flag:
            self.label.setText("Data is collected")
            flag=True
        else:
            self.label.setText("Data is not collected")
            flag=False
    def save(self):
        np.save('vib_{}.npy'.format(datetime.datetime.now().strftime("%m_%d_%H_%M_%S")), \
               np.array(L))
        
class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None, width=15, height=4, dpi=100):
        super().__init__( Figure(figsize=(width, height), dpi=dpi) )
        self.setParent(parent)
        self.data = [0 for i in range(25)]
    def plotc(self):
        global flag
        new = data_collect()
        self.data = self.data[1:] + [new]
        fig = self.figure.add_subplot(111)
        fig.clear()
        fig.plot(self.data, 'r-')
        fig.set_ylim(-2,2)
        fig.set_title('Vibration')
        self.draw()
        if flag:
            L.append(new)       

def data_collect():
    return np.random.randn()

In [3]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:57: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


SystemExit: 0

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
